In [ ]:
# default_exp classify_ions

In [ ]:
#export
import numpy as np
import pandas as pd
def collect_node_parameters(all_nodes, w_annot = True):
    ion2param2val = {}
    all_headers = set()
    for node in all_nodes:

        param2val = { "frac_mainclust" : node.frac_mainclust, "num_mainclusts" : node.num_mainclusts,
        "variance_fcs" : calc_variance_for_node(node), "num_children" : len(node.children), "fraction_consistent": node.fraction_consistent, "num_leaves" : len(node.leaves)}
        if w_annot:
            param2val.update({"positive_example" : node.positive_example, "ion" : node.name})
        if node.type == 'mod_seq_charge':
            if len(node.children) ==2:
                fcfc_diff = abs(node.children[0].fc - node.children[1].fc)
                param2val.update({"ms1_ms2_fcfc_diff" : fcfc_diff})

            for child in node.children:
                name_frac_mainclust = f"child_type_{child.type}_frac_mainclust"
                name_num_mainclusts = f"child_type_{child.type}_num_mainclusts"
                name_variance = f"child_type_{child.type}_variance_fcs"
                param2val.update({name_frac_mainclust: child.frac_mainclust, name_num_mainclusts : child.num_mainclusts, name_variance : calc_variance_for_node(child)})
        all_headers.update(param2val.keys())
        ion2param2val[node.name] = param2val

    return get_dataframe(ion2param2val, all_headers)

def get_dataframe(ion2param2val, all_headers):
    all_headers = list(all_headers)
    rows = [[ion2param2val.get(ion).get(header, np.nan) for header in all_headers] for ion in ion2param2val.keys()]
    df = pd.DataFrame(rows, columns = all_headers)
    return df


def calc_variance_for_node(node):
    fcs_children = [x.fc for x in node.children]
    #print(f"fcs children are {fcs_children}, variance is {np.var(fcs_children)}")

    return np.var(fcs_children)

In [ ]:
#export
import anytree

def get_positive_negative_samples(condpairtree,type):
    all_nodes = []
    for protein_node in condpairtree.children:
        if protein_node.num_mainclusts <3:
            continue
        if protein_node.fraction_consistent < 0.7:
            continue
        type_nodes = anytree.search.findall(protein_node, filter_=lambda node: node.type == type)
        for type_node in type_nodes:
            fcdiff = abs(type_node.fc - protein_node.fc)

            if fcdiff<0.3:
                type_node.positive_example = True
                all_nodes.append(type_node)
                continue
            if fcdiff>2:
                type_node.positive_example = False
                all_nodes.append(type_node)
    return all_nodes